In [1]:
import rospy
import jupyros as jr
import jupyros_action_client_patched as jr_patched

import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
from assignment_2_2023.msg import PlanningGoal, PlanningAction
from monitoring_package.msg import poseVelocity

In [3]:
rospy.init_node('jupyter_node')
r = rospy.Rate(10) # 10hz

In [4]:
goals_achieved = 0
goals_cancelled = 0

x_goal, y_goal = (1, 1)

def action_feedback_callback(e):
    global goals_cancelled, goals_achieved
    if (e.stat.strip() == 'Target cancelled!'):
        goals_cancelled+=1
    elif (e.stat.strip() == 'Target reached!'):
        goals_achieved+=1
        
def sent_goal_callback(m):
    global x_goal, y_goal
    x_goal = m.target_pose.pose.position.x
    y_goal = m.target_pose.pose.position.y
    


In [5]:
jr_patched.action_client('reaching_goal', PlanningAction, PlanningGoal, callbacks={'feedback_cb': action_feedback_callback, 'last_sent_goal_cb': sent_goal_callback})

[INFO] [1716830915.218699, 4178.721000]: [REACHING_GOAL] Waiting for action server.
[INFO] [1716830915.263731, 4178.769000]: [REACHING_GOAL] Connection to server successful.


In [6]:
print(goals_achieved, goals_cancelled)
print(x_goal, y_goal)

0 0
1 1


In [7]:
import matplotlib.pyplot as plt
import rospy
import tf
from turtlesim.msg import Pose
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation

class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.ln2 = plt.plot([], [], 'bs')
        self.x_data, self.y_data = [] , []
        self.x_goal, self.y_goal = [], []

    def plot_init(self):
        self.ax.set_xlim(0, 10)
        self.ax.set_ylim(0, 10)
        return self.ln, self.ln2

    def poseCallback(self, pV):
        print('called')
        global x_arr, y_arr

        if (len(self.x_data) > 0 and len(self.y_data) > 0):
            if (((self.x_data[-1]-pV.x)**2 + (self.y_data[-1]-pV.y)**2)**0.5 < 0.1):

#                 self.x_data.append(pV.x)
                self.x_data = [pV.x]
                self.y_data = [pV.y]
#                 self.y_data.append(pV.y)


        else:
#             self.x_data.append(pV.x)
#             self.y_data.append(pV.y)
            self.x_data = [pV.x]
            self.y_data = [pV.y]
        global x_goal, y_goal
        self.x_goal, self.y_goal = 5, 5

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        self.ln2.set_data(self.x_goal, self.y_goal)
        return self.ln, self.ln2



In [8]:

#     plt.cla()
#     ax.set_xlim([-10, 10])
#     ax.set_ylim([-10, 10])
#     ax.plot(x_arr, y_arr, linestyle='none', marker='.')
#     plt.pause(0.05)

    

In [9]:
# Create the animation
%matplotlib widget
# Create a new figure and add a subplot

vis = Visualiser()
jr.subscribe("/current_pose_velocity_publisher", poseVelocity, callback=vis.poseCallback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
print(vis.x_goal)

In [ ]:
# print(y_arr)

In [ ]:
# print(goals_achieved)
